In [1]:
from IPython.display import Code
import inspect

import sys
# Own code
sys.path.insert(0, './') # add local sources
from internal.GBP import GaussianState, VariableNode, FactorNode, FactorGraph


# On fitting a line with Gaussian Belief Propagation

This blog post is just me trying to reproduce the results of the excellent blog post: [A visual introduction to Gaussian Belief Propagation](https://gaussianbp.github.io/).
Please check this site out if you want to have a general introduction.

In here I want to focus on the implementational details.
This implementation does NOT focus on performance. 
Instead it focus on code readability so that it is easier to understand what's going on.
The main purpose I'm doing this, is to learn for myself. As reading stuff and actually replicating it, is something different.
To give credit where it belongs: I got some ideas on how to implement this from [this repository](https://github.com/joeaortiz/gbp).

In this blog article I go step by step trough the code. Nevertheless, you can find the complete implementation in my [repository](https://github.com/Schwarzstift/blog).
If thats of interest to you, go ahead and read more :)

## What needs to be implemented?

Before I start implementing stuff, I like to step back and look at the bigger picture.
What do we need for a basic GBP algorithm to work?

My answer to this question is that we need roughly 4 classes:
- a gaussian state container
- a variable node containing
    - the current state belief of the node
    - a function to update the belief
    - a function to compute the message from a node to a factor
- factor node containing
    - the factor of the factor node
    - a function to compute the message from a factor to a node
- factor graph
    - which wraps everything up

Lets first implement the basic structure of a Gaussian Belief Propagation algorithm.

## Gaussian State

As the name Gaussian Belief Propagation implies is the bases of all the Gaussian distribution.
Hence we start with implementing a tiny helper class to keep the mean vector and covariance matrix close together.
As pointed out in this [blog post](https://gaussianbp.github.io/), the canonical form offers some computational advantages above the moments form.
Hence we will also keep the state in this form.

In [2]:
lines = "class " + GaussianState.__name__ + "\n"
lines += inspect.getsource(GaussianState.__init__)
Code(lines, language="python")

class GaussianState
    def __init__(self, dimensionality: int, eta: np.ndarray = None, lam: np.ndarray = None):
        """
        Initialize a gaussian state
        :param dimensionality: number of dimensions of the state
        :param eta: canonical form of mean
        :param lam: canonical form of the covariance matrix
        """
        self.dim = dimensionality
        if eta is not None and len(eta) == self.dim:
            self.eta = eta
        else:
            self.eta = np.zeros(self.dim)

        if lam is not None and lam.shape == (self.dim, self.dim):
            self.lam = lam
        else:
            self.lam = np.zeros([self.dim, self.dim])

## Variable Node 

The variable node represent the random variables in your factor graph.
As we use Gaussian Belief Propagation (GBP), each variable is assumed to be normal distributed.

NOTE:
- A variable node can contain a random variable vector (implied by it's dimensions)!

In [3]:
lines = "class " + VariableNode.__name__ + "\n"
lines += inspect.getsource(VariableNode.__init__)
Code(lines, language="python")

class VariableNode
    def __init__(self, dimensions: int, idx: int):
        """
        Initialize the belief and prior with gaussian states
        :param dimensions: the dimensions of the gaussian state
        :param idx: an id unique over all variable nodes.
        """
        self.idx = idx
        self.belief = GaussianState(dimensions)
        self.prior = GaussianState(dimensions)
        self.dimensions = dimensions
        self.adj_factors = []  # will be filled by Factors on creation

        self.mu = np.zeros(dimensions)  # for debug/output purpose
        self.sigma = np.zeros([dimensions, dimensions])  # for debug/output purpose

### Belief update

Here I want to cite the Appendix B from [here](https://gaussianbp.github.io/):
> To compute the belief at a variable node, you take the product of incoming messages from all adjacent factors: 
> $$b_i(x_i)= \prod_{s \in N(i)} m_{f_s \rightarrow x_i}, $$
> where N(i) denotes the neighbours of nodes i. A Gaussian message has the canonical form: 
> $$ m = N^{-1}(x,\eta,\Lambda) \propto exp(-\frac{1}{2} x^\intercal \Lambda x+\eta^\intercal x), $$
> and so taking products of these messages is equivalent to summing the respective information vectors and precision matrices. The belief parameters $\eta_{b_i}$ and $\Lambda_{b_i}$ are therefore:
> $$ \eta_{b_i} = \sum_{s \in N(i)} \eta_{f_s \rightarrow x_i} \text{ and }  \Lambda_{b_i} = \sum_{s \in N(i)} \Lambda_{f_s \rightarrow x_i}$$

As the last two equations suggest, the belief update is simply a sum over the incoming messages.

So let's implement this:

In [4]:
lines = "class " + VariableNode.__name__ + "\n"
lines += "    ...\n\n"
lines += inspect.getsource(VariableNode.update_belief)
Code(lines, language="python")

class VariableNode
    ...

    def update_belief(self):
        """
        Updates the belief of a variable node & sends a message to all adjacent factors
        """
        eta = self.prior.eta.copy()
        lam = self.prior.lam.copy()
        for factor in self.adj_factors:
            eta_message, lam_message = factor.get_message_for(self)
            eta += eta_message
            lam += lam_message

        self.belief.eta = eta
        self.belief.lam = lam
        self.sigma = np.linalg.inv(self.belief.lam)  # Just for debugging/output
        self.mu = self.sigma @ self.belief.eta  # Just for debugging/output

        # Send message with updated belief to adjacent factors
        for factor in self.adj_factors:
            eta_message, lam_message = factor.get_message_for(self)
            factor.receive_message_from(self.idx, eta - eta_message, lam - lam_message)

In the last step, the message from this variable node to each factor is updated.

It is necessary to subtract the factor message from the overall estimate in order to fulfill the following equation:
$$ \eta_{x_i \rightarrow f_j} = \sum_{s \in N(i)\setminus j} \eta_{f_s \rightarrow x_i} \text{ and }  \Lambda_{x_i \rightarrow f_j} = \sum_{s \in N(i)\setminus j} \Lambda_{f_s \rightarrow x_i}$$


## Factor Node

The factor node models the dependencies between the variable nodes as a joint gaussian distribution of all adjacent variable nodes.




In [5]:
lines = "class " + FactorNode.__name__ + "\n"
lines += inspect.getsource(FactorNode.__init__)
Code(lines, language="python")

class FactorNode
    def __init__(self, factor: GaussianState, adj_variable_nodes: List[VariableNode]):
        """
        Initialize internal variables & adds itself to all adjacent variable nodes
        :param factor: The gaussian factor of this factor node. Has to have the same dim. as all adj. variable nodes
        :param adj_variable_nodes: all variable nodes, which are adjacent to this factor node
        """
        self.factor = factor
        self.adj_variable_nodes = adj_variable_nodes
        self.adj_variable_messages = {}
        self.messages_to_adj_variables = {}

        number_of_conditional_variables = 0  # just as a sanity check
        for variable_node in self.adj_variable_nodes:
            variable_node.adj_factors.append(self)  # bind factor to variable
            number_of_conditional_variables += variable_node.dimensions
            self.adj_variable_messages[variable_node.idx] = variable_node.belief
            self.messages_to_adj_variables[variable_node.idx] = GaussianState(variable_node.dimensions)

        assert (self.factor.dim == number_of_conditional_variables)

### Receive messages from variable nodes


In [6]:
lines = "class " + FactorNode.__name__ + "\n"
lines += "    ...\n\n"
lines += inspect.getsource(FactorNode.receive_message_from)
Code(lines, language="python")

class FactorNode
    ...

    def receive_message_from(self, variable_idx: int, eta_message: np.ndarray, lam_message: np.ndarray):
        """
        Stores the new variable to factor message for the next iteration
        :param variable_idx: the idx of the variable where the message originated from
        :param eta_message: the eta part of the message
        :param lam_message: the lambda part of the message
        """
        self.adj_variable_messages[variable_idx].lam = lam_message
        self.adj_variable_messages[variable_idx].eta = eta_message

### Retriev factor to variable message from factor_node


In [7]:
lines = "class " + FactorNode.__name__ + "\n"
lines += "    ...\n\n"
lines += inspect.getsource(FactorNode.get_message_for)
Code(lines, language="python")

class FactorNode
    ...

    def get_message_for(self, variable_node: VariableNode):
        """
        Simple getter function to retrieve the correct message for a given variable node
        :param variable_node: message from this factor to variable_node
        :return: eta_message and lam_message
        """
        return self.messages_to_adj_variables[variable_node.idx].eta, self.messages_to_adj_variables[
            variable_node.idx].lam

### Compute factor to variable messages



In [8]:
lines = "class " + FactorNode.__name__ + "\n"
lines += "    ...\n\n"
lines += inspect.getsource(FactorNode.compute_outgoing_messages)
Code(lines, language="python")

class FactorNode
    ...

    def compute_outgoing_messages(self):
        """
        Computes all factor to variable messages for adjacent variable nodes.
        The results are stored in a class variable

        See this blog post Appendix B for the equations: https://gaussianbp.github.io/
        """
        current_variable_position = 0
        for variable_node in self.adj_variable_nodes:
            eta_factor, lam_factor = self.factor.eta.copy(), self.factor.lam.copy()

            # For every node take the product of factor and incoming messages
            current_factor_position = 0
            for other_variables in self.adj_variable_nodes:
                if variable_node != other_variables:
                    start = current_factor_position
                    end = current_factor_position + other_variables.dimensions
                    eta_factor[start:end] += self.adj_variable_messages[other_variables.idx].eta
                    lam_factor[start:end, start:end] += self.adj_variable_messages[other_variables.idx].lam
                current_factor_position += other_variables.dimensions

            # Marginalization to variable node
            # - First "reorder" variable_nodes's elements to the top
            cur_dims = variable_node.dimensions
            start = current_variable_position
            end = current_variable_position + cur_dims
            eta_a = eta_factor[start:end]  # information vector of variable_node
            eta_b = np.concatenate((eta_factor[:start], eta_factor[end:]))  # information vector of the rest

            lam_aa = lam_factor[start:end, start:end]
            lam_ab = np.hstack((lam_factor[start:end, :start], lam_factor[start:end, end:]))
            lam_ba = np.vstack((lam_factor[:start, start:end], lam_factor[end:, start:end]))
            lam_bb = np.block([[lam_factor[:start, :start], lam_factor[:start, end:]],
                               [lam_factor[end:, :start], lam_factor[end:, end:]]])

            # - Then marginalize according to https://ieeexplore.ieee.org/document/4020357
            new_message_eta = eta_a - lam_ab @ np.linalg.inv(lam_bb) @ eta_b
            new_message_lam = lam_aa - lam_ab @ np.linalg.inv(lam_bb) @ lam_ba
            self.messages_to_adj_variables[variable_node.idx].eta = new_message_eta
            self.messages_to_adj_variables[variable_node.idx].lam = new_message_lam

            current_variable_position += variable_node.dimensions

# Factor Graph